### Value at Risk - Historical Method

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf   
import datetime as dt  
from scipy.stats import norm

In [5]:
# Set time to a certain number of years

years = 15
end_date = dt.datetime.now()   
start_date = end_date - dt.timedelta(days=years*365)

In [6]:
# List of tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

In [7]:
# Using daily adjusted close prices for analysis
adj_close_df = pd.DataFrame()
for ticker in tickers:
    adj_close_df[ticker] = yf.download(ticker, start=start_date, end=end_date)['Adj Close']

adj_close_df.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GOOGL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMZN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


,AAPL,MSFT,GOOGL,AMZN,TSLA


In [ ]:
# Calculate daily log returns

log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()
log_returns.head()

In [ ]:
# Calculate expected return

def expected_return(weight, log_returns):
    return np.sum(weight * log_returns.mean())

In [ ]:
# Create equal weights for the portfolio
portfolio_value = 1000000
weights = np.array([1/len(tickers)] * len(tickers))
weights

In [ ]:
# Calculate historical portfolio returns
hist_returns = (log_returns * weights).sum(axis=1)
hist_returns.head()

In [ ]:
# Find X day VaR at 95% confidence level

days = 5

range_returns = hist_returns.rolling(window=days).sum()
range_returns = range_returns.dropna()
range_returns.head()


In [ ]:
# Specify confidence level and calculate VaR
confidence_level = 0.95
VaR= np.percentile(range_returns, 100 - (confidence_level * 100))*portfolio_value
VaR

In [ ]:
# Plot returns distribution

return_window = days
range_returns = hist_returns.rolling(window=return_window).sum()
range_returns = range_returns.dropna()  

range_returns_dollar = range_returns * portfolio_value 
range_returns_dollar = range_returns_dollar.dropna()

plt.hist(range_returns_dollar, bins=50, density=True)
plt.xlabel(f'{return_window}-Day Portfolio Returns ($)')
plt.ylabel('Frequency')
plt.title(f'Distribution of {return_window}-Day Portfolio Returns')
plt.axvline(x=VaR, color='r', linestyle='--', label=f'VaR at {int(confidence_level*100)}%: ${VaR:,.2f}')
plt.legend()
plt.show()
